In [23]:
import torch
from torch.nn.utils import prune

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

# from calculate_mask import calculate_mask
# from inverse_hessian import inverse_hessian
from input_prehooks import put_input_hooks

In [24]:
#DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'


#Load dataset
dataset = load_dataset('c4', 'en', streaming=True)
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True)
# Load genrator
generator = pipeline('text-generation', model="facebook/opt-125m")
# Create calibration data
calibration_data = []
for i, data in enumerate(iter(dataset['train'])):
    if i > 15:
        break
    tokenized = tokenizer.encode(data['text'], return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    calibration_data.append(tokenized)
calibration_data = torch.squeeze(torch.stack(calibration_data)).to(device=device)
calibration_data.dtype

In [25]:
# First, put in forward hooks
features = {}
put_input_hooks(model=model, features=features)

# Run calibration data through model at first to calculate features dictionary with
# input tensors to each intermediate layer
model(calibration_data)

# function to get module name from parameter name
def get_module_name(param_name):
    if param_name[-5:] == ".bias":
        return param_name[:-5], "bias"
    elif param_name[-7:] == ".weight":
        return param_name[:-7], "weight"
    else:
        return None, None

In [26]:
features['model.decoder.embed_tokens']

(tensor([[    2, 48290,  7130,  ...,     1,     1,     1],
         [    2, 48763,    11,  ...,     1,     1,     1],
         [    2,   597, 14189,  ...,     1,     1,     1],
         ...,
         [    2,   100,    33,  ...,     1,     1,     1],
         [    2, 16750, 37123,  ...,     1,     1,     1],
         [    2,   113,   387,  ...,     6,  4753,     4]]),)

In [27]:
def calculate_mask(
    W,
    H_inv,
    p,
    B,
    Bs,
    ):

    # Get the number of rows and columns in W

    (d_row, d_col) = W.shape

    # Initialize the pruning mask M and block quantization errors E to all zeros

    M = torch.zeros(d_row, d_col, dtype=torch.bool)
    E = torch.zeros(d_row, B)

    # only need to calculate w_square and h_square once
    # Loop over blocks of columns of W (as specified by B)

    for i in range(0, d_col, B):

        # Loop over columns within a block

        for j in range(i, min(i + B, d_col)):

            # If j is a multiple of Bs, prune a portion of the weights

            if j % Bs == 0:

                # Get the mask for the largest (1 - p)% of weights based on squared value and inverse hessian

                # ASTERISK: prune_values is matrix of w^2/H^(-1)_cc

                # Finding respective sections of hessian and weights matrix
                w_square_section = torch.square(W[:, j:j + Bs])
                h_square_section = torch.square(H_inv[j:j + Bs, j:j
                        + Bs]).diag()  # 1 dimensional vector

                # getting the prune values matrix from W and H^-1 sections
                prune_values = w_square_section \
                    / h_square_section.unsqueeze(0)

                #calulating cutoff for the weights
                cutoff_value = torch.kthvalue(prune_values, int((1 - p)
                        * d_row), dim=0)[0]

                #getting the final mask
                mask = prune_values > cutoff_value

                #masking
                M[:, j:j + Bs] = mask

            # Calculate the pruning error for this column

            E[:, j - i] = W[:, j] / H_inv[j, j]

            # Freeze the weights that are not pruned by multiplying by the pruning mask
            # Invert mask (~M equivalent to 1 - M < might be -(M + 1))

            E[:, j - i] = ~M[:, j] * E[:, j - i]

            # Update the weights in this block based on the pruning error and inverse hessian information

            W[:, j:i + B] -= torch.ger(E[:, j - i], H_inv[j, j:i + B])

        # Update all remaining weights

        W[:, i + B:] -= torch.matmul(E, H_inv[i:i + B, i + B:])

    # return mask

    return M


def inverse_hessian(X, epsilon=0.01):
    """
    Calculate the inverse of a positive-definite matrix using the Cholesky decomposition.
    Args:
    - X (torch.Tensor): dxn tensor
    - epsilon (float): small constant to prevent Hessian from being singular
    Returns:
    - torch.Tensor: inverted matrix
    """
    X = X.float()
    X_T = torch.transpose(X, 0, 1)
    identity = torch.eye(X.shape[0], dtype=torch.float64)
    H = 2 * (X @ X_T + (epsilon * identity))
    # print(torch.linalg.eig(H)[0])
    print(H.shape)
    print(f"num zeros in hessian: {torch.sum(H == 0)}")
    print(f"Determinant is {torch.linalg.det(H)}")
    print(f"Hessian Diagonal is {H.diag()}")
    H_inv = torch.inverse(H)
    
    #H_inv = torch.cholesky(H_inv).T
    H_inv = torch.lu(H_inv)[0].T
    
    return H_inv


# Re-load model with pre-trained head
model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True)

# make a dictionary to access module by name
model_lookup_dict = {}
for module_name, module_iter in model.named_modules():
    model_lookup_dict[module_name] = module_iter

EPSILON = 1e-8
SPARSENESS = .2
B = 32
Bs = 16

# Using calibration data (inputs to each intermediate weight layer)
# Iterate through named parameters, calculate inverse hessian and calculate mask
with torch.no_grad():
    for name, param in model.named_parameters():
        module_name, param_type = get_module_name(name)

        # apply to weight and bias layers
        if param_type == "weight" or param_type == "bias":
            # input to parameter
            layer_input = features[module_name]
            # calculate inverse hessian
            inv_hess = inverse_hessian(torch.transpose(layer_input[0], 0, 1), epsilon=EPSILON)

            # calculate mask
            mask = calculate_mask(W=param, H_inv=inv_hess, p=SPARSENESS, B=B, Bs=Bs)
            
            # get module from lookup dictionary by module name
            module = model_lookup_dict[module_name]
            # apply mask
            prune.custom_from_mask(module=module, name=param_type, mask=mask)
        break

torch.Size([512, 512])
num zeros in hessian: 0
Determinant is inf
Hessian Diagonal is tensor([1.2800e+02, 2.1211e+10, 4.4277e+09, 4.0502e+09, 6.9439e+09, 6.9146e+09,
        3.8837e+09, 2.8223e+09, 3.6256e+09, 5.8754e+09, 8.0317e+09, 7.6373e+09,
        1.8665e+10, 7.1441e+09, 9.2101e+08, 4.8976e+09, 8.7763e+09, 8.6306e+09,
        2.9042e+08, 6.2431e+08, 2.2291e+09, 1.1374e+09, 7.0100e+09, 7.1680e+09,
        4.2909e+09, 5.3398e+09, 1.1318e+10, 2.9786e+09, 4.3516e+09, 6.2345e+09,
        8.3428e+09, 8.5852e+09, 5.9779e+09, 6.3430e+09, 1.2358e+09, 4.2279e+08,
        5.6564e+09, 1.3808e+09, 6.9630e+09, 1.1036e+10, 1.3557e+10, 4.8947e+09,
        4.9650e+09, 2.3144e+09, 9.7131e+09, 7.7709e+09, 3.7450e+09, 5.0588e+09,
        5.7672e+09, 1.3213e+10, 5.6264e+09, 3.1772e+08, 5.2932e+09, 7.2946e+09,
        5.2837e+09, 6.2821e+09, 7.5086e+09, 3.7343e+09, 1.7969e+09, 3.6061e+09,
        5.5131e+09, 2.1430e+08, 6.1203e+09, 3.1141e+09, 5.3713e+09, 8.1849e+08,
        6.7303e+08, 6.8726e+09, 7.

RuntimeError: expected scalar type Float but found Double